In [5]:
from datasets import load_dataset
dataset = load_dataset("hellaswag")
dataset


/Users/vince/miniforge3/envs/cse251u/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for hellaswag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hellaswag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
        num_rows: 39905
    })
    test: Dataset({
        features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
        num_rows: 10003
    })
    validation: Dataset({
        features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
        num_rows: 10042
    })
})

In [6]:
import pandas as pd

train_df = pd.DataFrame(dataset['train'])

In [39]:
sampled_df = train_df.groupby('activity_label').apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)
if len(sampled_df) > 10:
    sampled_df = sampled_df.sample(10, random_state=42).reset_index(drop=True)
sampled_df

/var/folders/dv/lndb9rrs2kv2v5g8cyfyx0wr0000gn/T/ipykernel_31076/1460773718.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = train_df.groupby('activity_label').apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)


,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label
0,34590,Camel ride,A man is seen holding a small child while look...,the two,A man is seen holding a small child while look...,[are shown in several more shots climbing on t...,activitynet~v_pPGniqIBs-U,train,indomain,3
1,2539,Drum corps,Text appears on the screen with a link. People...,it,Text appears on the screen with a link. People...,"[then cuts to a news anchor with a laptop., en...",activitynet~v_USaP2UCPBEc,train,indomain,2
2,8031,Slacklining,A man stretches a rope between two trees.,he,A man stretches a rope between two trees. he,[gets on the rope and prepares to start a race...,activitynet~v_73AGD3RWPEw,train,indomain,1
3,46953,Computers and Electronics,[header] How to change a computer hard drive d...,,[header] How to change a computer hard drive d...,[[step] Most small-sized hard drives need an o...,wikihow~77249,train,indomain,1
4,18878,Horseback riding,.,the lady in blue vest and brown jacket,. the lady in blue vest and brown jacket,[has some more practice punches and then takes...,activitynet~v_W50sQxSWDwM,train,indomain,1
5,16308,Building sandcastles,A group of kids is building a moat filled with...,we,A group of kids is building a moat filled with...,[see fireworks going off in the background as ...,activitynet~v_TK5FnYshy10,train,indomain,3
6,36087,Putting on makeup,Young girl puts black mascara on her eyelashes.,she,Young girl puts black mascara on her eyelashes...,"[then touches her eyelids, her eyebrows, and h...",activitynet~v_F51cKkjt6tk,train,indomain,2
7,15105,Work World,[header] How to explain a termination in a job...,,[header] How to explain a termination in a job...,[If you can just paint a picture of the situat...,wikihow~24971,train,indomain,3
8,4424,Playing saxophone,A magazine about saxophones is shown just befo...,multiple screens,A magazine about saxophones is shown just befo...,[appear with features to replace the ones show...,activitynet~v_oTKYejnHLtE,train,indomain,2
9,177,Smoking a cigarette,A camera pans up to a man sitting and smoking ...,the man smoking cigarettes,A camera pans up to a man sitting and smoking ...,[continues shoveling the debris along the grou...,activitynet~v_0acEl97ZBME,train,indomain,3


### Default hyperparameters
Testing LLaMA 3 with default hyperparameters.
```
temperature = 0.8
top_k = 40
top_p = 0.9
```

In [57]:
import os
from langchain_community.llms import Ollama
from IPython.display import clear_output
from pprint import pprint
import pandas as pd
from datasets import load_dataset


# Initialize the Llama 3 model using LangChain and Ollama with hyperparameters
# llm = Ollama(model="llama3", temperature=1.2, top_k=50, top_p=0.95)

llm = Ollama(model="llama3", temperature=0.8, top_k=40, top_p=0.9)

# Function to format the HellaSwag example for multiple-choice
def format_example(example):
    question = example['ctx']
    choices = example['endings']
    return question, choices, int(example['label'])  # Ensure label is an integer

# Function to extract the choice from the model's response
def extract_choice(response):
    if "[A]" in response:
        return 0
    elif "[B]" in response:
        return 1
    elif "[C]" in response:
        return 2
    elif "[D]" in response:
        return 3
    else:
        return -1  # Indicates an error or unrecognized format

# Function to test a single example multiple times
def test_single_example_multiple_times(idx, num_samples=5):
    example = sampled_df.iloc[idx]
    question, choices, label = format_example(example)
    prompt = "Your task is to think carefully through a reasoning problem.\n"
    prompt += "Respond only with [A], [B], [C], or [D]\n"
    prompt += f"Question: {question}\n"
    labels = ['A', 'B', 'C', 'D']
    for i, choice in enumerate(choices):
        prompt += f"Choice [{labels[i]}]: {choice}\n"
    prompt += "Which choice is the best answer?"

    responses = []
    for _ in range(num_samples):
        response = llm.invoke(prompt)
        predicted_choice = extract_choice(response)
        responses.append(predicted_choice)
    

    correct_responses = sum(1 for choice in responses if choice == label)
    response_distribution = {labels[i]: responses.count(i) for i in range(4)}
    
    clear_output(wait=True)
    print(f"Activity: {example['activity_label']}")
    print(f"Question {idx + 1}:")
    pprint(prompt)
    print(f"\nCorrect Choice: {labels[label]}")
    print(f"Number of Correct Responses: {correct_responses}/{num_samples}")
    print("Response Distribution:")
    for label, count in response_distribution.items():
        percentage = (count / num_samples) * 100
        print(f"{label}: {percentage:.2f}%")

# Test a single example by index and sample multiple times
test_single_example_multiple_times(0, num_samples=10)  # Change the index to test different examples


Activity: Camel ride
Question 1:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man is seen holding a small child while looking and smiling to '
 'the camera. The two then climb on a camel and ride around while waving to '
 'the camera. the two\n'
 'Choice [A]: are shown in several more shots climbing on the camel and '
 'walking back.\n'
 'Choice [B]: end up getting off and walking around their ride.\n'
 'Choice [C]: continue riding until the baby pulls a dog down off the camel '
 'and the man leaves.\n'
 'Choice [D]: ride back together with the woman leading them in front.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [58]:
test_single_example_multiple_times(1, num_samples=10)

Activity: Drum corps
Question 2:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Text appears on the screen with a link. People in uniform march '
 'down the street. it\n'
 'Choice [A]: then cuts to a news anchor with a laptop.\n'
 'Choice [B]: ends with a lead in footage of the participants walking and '
 'cheering.\n'
 'Choice [C]: cuts to the crowd looking on.\n'
 'Choice [D]: screens an ad for the snow camp, with its picture of a cabin.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 2/10
Response Distribution:
A: 50.00%
B: 30.00%
C: 20.00%
D: 0.00%


In [59]:
test_single_example_multiple_times(2, num_samples=10)

Activity: Slacklining
Question 3:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man stretches a rope between two trees. he\n'
 'Choice [A]: gets on the rope and prepares to start a race.\n'
 'Choice [B]: then balances on the rope as he walks, sits, and bounces on it.\n'
 'Choice [C]: begins walking forward in the path.\n'
 'Choice [D]: puts a shoe on to hold onto the rope.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 3/10
Response Distribution:
A: 0.00%
B: 30.00%
C: 60.00%
D: 10.00%


In [60]:
test_single_example_multiple_times(3, num_samples=10)

Activity: Computers and Electronics
Question 4:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to change a computer hard drive disk [title] If your '
 'hard drive is still working, make sure you make recovery disk or image by '
 'using backup or recovery software. [step] If your drive has already failed, '
 'hopefully you have your important files saved somewhere as backup. [title] '
 'Obtain a new hard drive.\n'
 'Choice [A]: [step] Most small-sized hard drives need an old hard drive to go '
 'into recovery. However, it may take a long time to get old hard drives to '
 'get back to their original capacity.\n'
 'Choice [B]: [step] Make sure you have the right type of hard drive for your '
 'computer. Generally speaking, you want to replace your original drive with a '
 'similar drive to make the process easier and simpler.\n'
 'Choice [C]: [step] An external hard drive, such as a wineing or tonneau

In [55]:
test_single_example_multiple_times(3, num_samples=10)

Question 4:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to change a computer hard drive disk [title] If your '
 'hard drive is still working, make sure you make recovery disk or image by '
 'using backup or recovery software. [step] If your drive has already failed, '
 'hopefully you have your important files saved somewhere as backup. [title] '
 'Obtain a new hard drive.\n'
 'Choice [A]: [step] Most small-sized hard drives need an old hard drive to go '
 'into recovery. However, it may take a long time to get old hard drives to '
 'get back to their original capacity.\n'
 'Choice [B]: [step] Make sure you have the right type of hard drive for your '
 'computer. Generally speaking, you want to replace your original drive with a '
 'similar drive to make the process easier and simpler.\n'
 'Choice [C]: [step] An external hard drive, such as a wineing or tonneau '
 '2200. [title] Insert the hard d

In [61]:
test_single_example_multiple_times(4, num_samples=10)

Activity: Horseback riding
Question 5:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: . the lady in blue vest and brown jacket\n'
 'Choice [A]: has some more practice punches and then takes one hand grab her '
 'elbow and go straight into a crouch and to the side to do the same and then '
 'grab her armpit to throw the punch.\n'
 'Choice [B]: is sitting on a brown horse, while a man is standing beside the '
 'horse, holding another horse.\n'
 'Choice [C]: holds an orange handled knife and the lady in the green vest '
 'holds an orange handled knife with the blue handled handle.\n'
 'Choice [D]: is now standing at the mountain top preparing to get started '
 'putting bungee jumps on the ground.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 3/10
Response Distribution:
A: 0.00%
B: 30.00%
C: 0.00%
D: 70.00%


In [62]:
test_single_example_multiple_times(5, num_samples=10)

Activity: Building sandcastles
Question 6:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A group of kids is building a moat filled with water around a sand '
 'castle. A little girl with floaters on pour water in the moat. we\n'
 'Choice [A]: see fireworks going off in the background as the kids continue '
 'to build.\n'
 'Choice [B]: see an opening title screen.\n'
 'Choice [C]: see children on the other side avoiding the water castle.\n'
 'Choice [D]: see kids across the beach working on their castles in the wet '
 'sand.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [63]:
test_single_example_multiple_times(6, num_samples=10)

Activity: Putting on makeup
Question 7:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Young girl puts black mascara on her eyelashes. she\n'
 'Choice [A]: then touches her eyelids, her eyebrows, and her cheeks.\n'
 'Choice [B]: then applies painted and painted eyeliner to her eye lashes.\n'
 'Choice [C]: looks into the large mirror.\n'
 'Choice [D]: puts red colored eye lashes, then does eye shadow.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 1/10
Response Distribution:
A: 10.00%
B: 20.00%
C: 10.00%
D: 60.00%


In [64]:
test_single_example_multiple_times(7, num_samples=10)

Activity: Work World
Question 8:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to explain a termination in a job interview [title] '
 'Be honest. [step] When your interviewer asks you why you left your last job, '
 'the best thing you can do is be honest. Making up stories will only make you '
 'seem irresponsible and untrustworthy.\n'
 'Choice [A]: If you can just paint a picture of the situation, you will '
 'hardly convince your interviewer. Stick to facts you can use to bridge the '
 'gap between your actions and those of the employer.\n'
 "Choice [B]: Don't exaggerate what might have happened or be vague about the "
 "specific circumstances that led to you leaving. [substeps] Don't contact one "
 "of your supervisors to tell them you've terminated your job.\n"
 'Choice [C]: You will seem like a liar by having an inaccurate job '
 'description and a reputation of being a failure. Choose facts 

In [65]:
test_single_example_multiple_times(8, num_samples=10)

Activity: Playing saxophone
Question 9:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A magazine about saxophones is shown just before a pair of hands '
 'are shown playing a saxophone. multiple screens\n'
 'Choice [A]: appear with features to replace the ones shown on the tv '
 'screen.\n'
 'Choice [B]: show the two hands playing as they play one more tune on the '
 'saxophone.\n'
 'Choice [C]: begin to appear, showing the hands playing various notes on the '
 'screen.\n'
 'Choice [D]: flash and a keyboard appears with the word " connect " and an '
 'arrow going down.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 1/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 10.00%
D: 90.00%


In [66]:
test_single_example_multiple_times(9, num_samples=10)

Activity: Smoking a cigarette
Question 10:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A camera pans up to a man sitting and smoking a cigarette. Another '
 'man walks over with a cart and begins digging through trash. the man smoking '
 'cigarettes\n'
 'Choice [A]: continues shoveling the debris along the ground.\n'
 'Choice [B]: walks back with the cart while the second man continues to '
 'shovel the dirt.\n'
 'Choice [C]: takes the cigarette out of his mouth and finally spits it out.\n'
 'Choice [D]: begins talking to the homeless man about how life sucks and how '
 'he can relate to the homeless man.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 4/10
Response Distribution:
A: 0.00%
B: 40.00%
C: 20.00%
D: 40.00%


### Low top k 
Conservative approach
```
temperature = 0.8
top_k = 10
top_p = 0.9
```


In [71]:
llm = Ollama(model="llama3", temperature=0.8, top_k=10, top_p=0.9)
test_single_example_multiple_times(0, num_samples=10)

Activity: Camel ride
Question 1:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man is seen holding a small child while looking and smiling to '
 'the camera. The two then climb on a camel and ride around while waving to '
 'the camera. the two\n'
 'Choice [A]: are shown in several more shots climbing on the camel and '
 'walking back.\n'
 'Choice [B]: end up getting off and walking around their ride.\n'
 'Choice [C]: continue riding until the baby pulls a dog down off the camel '
 'and the man leaves.\n'
 'Choice [D]: ride back together with the woman leading them in front.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [72]:
test_single_example_multiple_times(1, num_samples=10)

Activity: Drum corps
Question 2:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Text appears on the screen with a link. People in uniform march '
 'down the street. it\n'
 'Choice [A]: then cuts to a news anchor with a laptop.\n'
 'Choice [B]: ends with a lead in footage of the participants walking and '
 'cheering.\n'
 'Choice [C]: cuts to the crowd looking on.\n'
 'Choice [D]: screens an ad for the snow camp, with its picture of a cabin.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 1/10
Response Distribution:
A: 50.00%
B: 40.00%
C: 10.00%
D: 0.00%


In [73]:
test_single_example_multiple_times(2, num_samples=10)

Activity: Slacklining
Question 3:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man stretches a rope between two trees. he\n'
 'Choice [A]: gets on the rope and prepares to start a race.\n'
 'Choice [B]: then balances on the rope as he walks, sits, and bounces on it.\n'
 'Choice [C]: begins walking forward in the path.\n'
 'Choice [D]: puts a shoe on to hold onto the rope.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 3/10
Response Distribution:
A: 0.00%
B: 30.00%
C: 60.00%
D: 10.00%


In [74]:
test_single_example_multiple_times(3, num_samples=10)

Activity: Computers and Electronics
Question 4:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to change a computer hard drive disk [title] If your '
 'hard drive is still working, make sure you make recovery disk or image by '
 'using backup or recovery software. [step] If your drive has already failed, '
 'hopefully you have your important files saved somewhere as backup. [title] '
 'Obtain a new hard drive.\n'
 'Choice [A]: [step] Most small-sized hard drives need an old hard drive to go '
 'into recovery. However, it may take a long time to get old hard drives to '
 'get back to their original capacity.\n'
 'Choice [B]: [step] Make sure you have the right type of hard drive for your '
 'computer. Generally speaking, you want to replace your original drive with a '
 'similar drive to make the process easier and simpler.\n'
 'Choice [C]: [step] An external hard drive, such as a wineing or tonneau

In [75]:
test_single_example_multiple_times(4, num_samples=10)

Activity: Horseback riding
Question 5:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: . the lady in blue vest and brown jacket\n'
 'Choice [A]: has some more practice punches and then takes one hand grab her '
 'elbow and go straight into a crouch and to the side to do the same and then '
 'grab her armpit to throw the punch.\n'
 'Choice [B]: is sitting on a brown horse, while a man is standing beside the '
 'horse, holding another horse.\n'
 'Choice [C]: holds an orange handled knife and the lady in the green vest '
 'holds an orange handled knife with the blue handled handle.\n'
 'Choice [D]: is now standing at the mountain top preparing to get started '
 'putting bungee jumps on the ground.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 5/10
Response Distribution:
A: 10.00%
B: 50.00%
C: 0.00%
D: 40.00%


In [76]:
test_single_example_multiple_times(6, num_samples=10)

Activity: Putting on makeup
Question 7:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Young girl puts black mascara on her eyelashes. she\n'
 'Choice [A]: then touches her eyelids, her eyebrows, and her cheeks.\n'
 'Choice [B]: then applies painted and painted eyeliner to her eye lashes.\n'
 'Choice [C]: looks into the large mirror.\n'
 'Choice [D]: puts red colored eye lashes, then does eye shadow.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 30.00%
B: 0.00%
C: 0.00%
D: 70.00%


In [77]:
test_single_example_multiple_times(7, num_samples=10)

Activity: Work World
Question 8:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to explain a termination in a job interview [title] '
 'Be honest. [step] When your interviewer asks you why you left your last job, '
 'the best thing you can do is be honest. Making up stories will only make you '
 'seem irresponsible and untrustworthy.\n'
 'Choice [A]: If you can just paint a picture of the situation, you will '
 'hardly convince your interviewer. Stick to facts you can use to bridge the '
 'gap between your actions and those of the employer.\n'
 "Choice [B]: Don't exaggerate what might have happened or be vague about the "
 "specific circumstances that led to you leaving. [substeps] Don't contact one "
 "of your supervisors to tell them you've terminated your job.\n"
 'Choice [C]: You will seem like a liar by having an inaccurate job '
 'description and a reputation of being a failure. Choose facts 

In [78]:
test_single_example_multiple_times(8, num_samples=10)

Activity: Playing saxophone
Question 9:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A magazine about saxophones is shown just before a pair of hands '
 'are shown playing a saxophone. multiple screens\n'
 'Choice [A]: appear with features to replace the ones shown on the tv '
 'screen.\n'
 'Choice [B]: show the two hands playing as they play one more tune on the '
 'saxophone.\n'
 'Choice [C]: begin to appear, showing the hands playing various notes on the '
 'screen.\n'
 'Choice [D]: flash and a keyboard appears with the word " connect " and an '
 'arrow going down.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [79]:
test_single_example_multiple_times(9, num_samples=10)

Activity: Smoking a cigarette
Question 10:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A camera pans up to a man sitting and smoking a cigarette. Another '
 'man walks over with a cart and begins digging through trash. the man smoking '
 'cigarettes\n'
 'Choice [A]: continues shoveling the debris along the ground.\n'
 'Choice [B]: walks back with the cart while the second man continues to '
 'shovel the dirt.\n'
 'Choice [C]: takes the cigarette out of his mouth and finally spits it out.\n'
 'Choice [D]: begins talking to the homeless man about how life sucks and how '
 'he can relate to the homeless man.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 6/10
Response Distribution:
A: 0.00%
B: 20.00%
C: 20.00%
D: 60.00%


### Low top k, low top p 
More conservative approach
```
temperature = 0.8
top_k = 10
top_p = 0.5
```


In [90]:
llm = Ollama(model="llama3", temperature=0.8, top_k=10, top_p=0.5)
test_single_example_multiple_times(0, num_samples=10)

Activity: Camel ride
Question 1:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man is seen holding a small child while looking and smiling to '
 'the camera. The two then climb on a camel and ride around while waving to '
 'the camera. the two\n'
 'Choice [A]: are shown in several more shots climbing on the camel and '
 'walking back.\n'
 'Choice [B]: end up getting off and walking around their ride.\n'
 'Choice [C]: continue riding until the baby pulls a dog down off the camel '
 'and the man leaves.\n'
 'Choice [D]: ride back together with the woman leading them in front.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [91]:
test_single_example_multiple_times(1, num_samples=10)

Activity: Drum corps
Question 2:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Text appears on the screen with a link. People in uniform march '
 'down the street. it\n'
 'Choice [A]: then cuts to a news anchor with a laptop.\n'
 'Choice [B]: ends with a lead in footage of the participants walking and '
 'cheering.\n'
 'Choice [C]: cuts to the crowd looking on.\n'
 'Choice [D]: screens an ad for the snow camp, with its picture of a cabin.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 2/10
Response Distribution:
A: 80.00%
B: 0.00%
C: 20.00%
D: 0.00%


In [92]:
test_single_example_multiple_times(2, num_samples=10)

Activity: Slacklining
Question 3:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man stretches a rope between two trees. he\n'
 'Choice [A]: gets on the rope and prepares to start a race.\n'
 'Choice [B]: then balances on the rope as he walks, sits, and bounces on it.\n'
 'Choice [C]: begins walking forward in the path.\n'
 'Choice [D]: puts a shoe on to hold onto the rope.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 5/10
Response Distribution:
A: 0.00%
B: 50.00%
C: 50.00%
D: 0.00%


In [93]:
test_single_example_multiple_times(3, num_samples=10)

Activity: Computers and Electronics
Question 4:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to change a computer hard drive disk [title] If your '
 'hard drive is still working, make sure you make recovery disk or image by '
 'using backup or recovery software. [step] If your drive has already failed, '
 'hopefully you have your important files saved somewhere as backup. [title] '
 'Obtain a new hard drive.\n'
 'Choice [A]: [step] Most small-sized hard drives need an old hard drive to go '
 'into recovery. However, it may take a long time to get old hard drives to '
 'get back to their original capacity.\n'
 'Choice [B]: [step] Make sure you have the right type of hard drive for your '
 'computer. Generally speaking, you want to replace your original drive with a '
 'similar drive to make the process easier and simpler.\n'
 'Choice [C]: [step] An external hard drive, such as a wineing or tonneau

In [94]:
test_single_example_multiple_times(4, num_samples=10)

Activity: Horseback riding
Question 5:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: . the lady in blue vest and brown jacket\n'
 'Choice [A]: has some more practice punches and then takes one hand grab her '
 'elbow and go straight into a crouch and to the side to do the same and then '
 'grab her armpit to throw the punch.\n'
 'Choice [B]: is sitting on a brown horse, while a man is standing beside the '
 'horse, holding another horse.\n'
 'Choice [C]: holds an orange handled knife and the lady in the green vest '
 'holds an orange handled knife with the blue handled handle.\n'
 'Choice [D]: is now standing at the mountain top preparing to get started '
 'putting bungee jumps on the ground.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 5/10
Response Distribution:
A: 0.00%
B: 50.00%
C: 0.00%
D: 50.00%


In [95]:
test_single_example_multiple_times(5, num_samples=10)

Activity: Building sandcastles
Question 6:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A group of kids is building a moat filled with water around a sand '
 'castle. A little girl with floaters on pour water in the moat. we\n'
 'Choice [A]: see fireworks going off in the background as the kids continue '
 'to build.\n'
 'Choice [B]: see an opening title screen.\n'
 'Choice [C]: see children on the other side avoiding the water castle.\n'
 'Choice [D]: see kids across the beach working on their castles in the wet '
 'sand.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [96]:
test_single_example_multiple_times(6, num_samples=10)

Activity: Putting on makeup
Question 7:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Young girl puts black mascara on her eyelashes. she\n'
 'Choice [A]: then touches her eyelids, her eyebrows, and her cheeks.\n'
 'Choice [B]: then applies painted and painted eyeliner to her eye lashes.\n'
 'Choice [C]: looks into the large mirror.\n'
 'Choice [D]: puts red colored eye lashes, then does eye shadow.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 10.00%
B: 0.00%
C: 0.00%
D: 90.00%


In [97]:
test_single_example_multiple_times(7, num_samples=10)

Activity: Work World
Question 8:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to explain a termination in a job interview [title] '
 'Be honest. [step] When your interviewer asks you why you left your last job, '
 'the best thing you can do is be honest. Making up stories will only make you '
 'seem irresponsible and untrustworthy.\n'
 'Choice [A]: If you can just paint a picture of the situation, you will '
 'hardly convince your interviewer. Stick to facts you can use to bridge the '
 'gap between your actions and those of the employer.\n'
 "Choice [B]: Don't exaggerate what might have happened or be vague about the "
 "specific circumstances that led to you leaving. [substeps] Don't contact one "
 "of your supervisors to tell them you've terminated your job.\n"
 'Choice [C]: You will seem like a liar by having an inaccurate job '
 'description and a reputation of being a failure. Choose facts 

In [98]:
test_single_example_multiple_times(8, num_samples=10)

Activity: Playing saxophone
Question 9:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A magazine about saxophones is shown just before a pair of hands '
 'are shown playing a saxophone. multiple screens\n'
 'Choice [A]: appear with features to replace the ones shown on the tv '
 'screen.\n'
 'Choice [B]: show the two hands playing as they play one more tune on the '
 'saxophone.\n'
 'Choice [C]: begin to appear, showing the hands playing various notes on the '
 'screen.\n'
 'Choice [D]: flash and a keyboard appears with the word " connect " and an '
 'arrow going down.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [99]:
test_single_example_multiple_times(9, num_samples=10)

Activity: Smoking a cigarette
Question 10:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A camera pans up to a man sitting and smoking a cigarette. Another '
 'man walks over with a cart and begins digging through trash. the man smoking '
 'cigarettes\n'
 'Choice [A]: continues shoveling the debris along the ground.\n'
 'Choice [B]: walks back with the cart while the second man continues to '
 'shovel the dirt.\n'
 'Choice [C]: takes the cigarette out of his mouth and finally spits it out.\n'
 'Choice [D]: begins talking to the homeless man about how life sucks and how '
 'he can relate to the homeless man.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


### Low temperature
Most conservative approach, same response every time
```
temperature = 0.0
top_k = 10
top_p = 0.5
```

In [100]:
llm = Ollama(model="llama3", temperature=0.0, top_k=10, top_p=0.5)
test_single_example_multiple_times(0, num_samples=10)

Activity: Camel ride
Question 1:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man is seen holding a small child while looking and smiling to '
 'the camera. The two then climb on a camel and ride around while waving to '
 'the camera. the two\n'
 'Choice [A]: are shown in several more shots climbing on the camel and '
 'walking back.\n'
 'Choice [B]: end up getting off and walking around their ride.\n'
 'Choice [C]: continue riding until the baby pulls a dog down off the camel '
 'and the man leaves.\n'
 'Choice [D]: ride back together with the woman leading them in front.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [101]:
test_single_example_multiple_times(1, num_samples=10)

Activity: Drum corps
Question 2:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Text appears on the screen with a link. People in uniform march '
 'down the street. it\n'
 'Choice [A]: then cuts to a news anchor with a laptop.\n'
 'Choice [B]: ends with a lead in footage of the participants walking and '
 'cheering.\n'
 'Choice [C]: cuts to the crowd looking on.\n'
 'Choice [D]: screens an ad for the snow camp, with its picture of a cabin.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 100.00%
B: 0.00%
C: 0.00%
D: 0.00%


In [102]:
test_single_example_multiple_times(2, num_samples=10)

Activity: Slacklining
Question 3:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man stretches a rope between two trees. he\n'
 'Choice [A]: gets on the rope and prepares to start a race.\n'
 'Choice [B]: then balances on the rope as he walks, sits, and bounces on it.\n'
 'Choice [C]: begins walking forward in the path.\n'
 'Choice [D]: puts a shoe on to hold onto the rope.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 0/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 100.00%
D: 0.00%


In [103]:
test_single_example_multiple_times(3, num_samples=10)

Activity: Computers and Electronics
Question 4:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to change a computer hard drive disk [title] If your '
 'hard drive is still working, make sure you make recovery disk or image by '
 'using backup or recovery software. [step] If your drive has already failed, '
 'hopefully you have your important files saved somewhere as backup. [title] '
 'Obtain a new hard drive.\n'
 'Choice [A]: [step] Most small-sized hard drives need an old hard drive to go '
 'into recovery. However, it may take a long time to get old hard drives to '
 'get back to their original capacity.\n'
 'Choice [B]: [step] Make sure you have the right type of hard drive for your '
 'computer. Generally speaking, you want to replace your original drive with a '
 'similar drive to make the process easier and simpler.\n'
 'Choice [C]: [step] An external hard drive, such as a wineing or tonneau

In [104]:
test_single_example_multiple_times(4, num_samples=10)

Activity: Horseback riding
Question 5:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: . the lady in blue vest and brown jacket\n'
 'Choice [A]: has some more practice punches and then takes one hand grab her '
 'elbow and go straight into a crouch and to the side to do the same and then '
 'grab her armpit to throw the punch.\n'
 'Choice [B]: is sitting on a brown horse, while a man is standing beside the '
 'horse, holding another horse.\n'
 'Choice [C]: holds an orange handled knife and the lady in the green vest '
 'holds an orange handled knife with the blue handled handle.\n'
 'Choice [D]: is now standing at the mountain top preparing to get started '
 'putting bungee jumps on the ground.\n'
 'Which choice is the best answer?')

Correct Choice: B
Number of Correct Responses: 0/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [105]:
test_single_example_multiple_times(5, num_samples=10)

Activity: Building sandcastles
Question 6:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A group of kids is building a moat filled with water around a sand '
 'castle. A little girl with floaters on pour water in the moat. we\n'
 'Choice [A]: see fireworks going off in the background as the kids continue '
 'to build.\n'
 'Choice [B]: see an opening title screen.\n'
 'Choice [C]: see children on the other side avoiding the water castle.\n'
 'Choice [D]: see kids across the beach working on their castles in the wet '
 'sand.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [106]:
test_single_example_multiple_times(6, num_samples=10)

Activity: Putting on makeup
Question 7:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Young girl puts black mascara on her eyelashes. she\n'
 'Choice [A]: then touches her eyelids, her eyebrows, and her cheeks.\n'
 'Choice [B]: then applies painted and painted eyeliner to her eye lashes.\n'
 'Choice [C]: looks into the large mirror.\n'
 'Choice [D]: puts red colored eye lashes, then does eye shadow.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [107]:
test_single_example_multiple_times(7, num_samples=10)

Activity: Work World
Question 8:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to explain a termination in a job interview [title] '
 'Be honest. [step] When your interviewer asks you why you left your last job, '
 'the best thing you can do is be honest. Making up stories will only make you '
 'seem irresponsible and untrustworthy.\n'
 'Choice [A]: If you can just paint a picture of the situation, you will '
 'hardly convince your interviewer. Stick to facts you can use to bridge the '
 'gap between your actions and those of the employer.\n'
 "Choice [B]: Don't exaggerate what might have happened or be vague about the "
 "specific circumstances that led to you leaving. [substeps] Don't contact one "
 "of your supervisors to tell them you've terminated your job.\n"
 'Choice [C]: You will seem like a liar by having an inaccurate job '
 'description and a reputation of being a failure. Choose facts 

In [108]:
test_single_example_multiple_times(8, num_samples=10)

Activity: Playing saxophone
Question 9:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A magazine about saxophones is shown just before a pair of hands '
 'are shown playing a saxophone. multiple screens\n'
 'Choice [A]: appear with features to replace the ones shown on the tv '
 'screen.\n'
 'Choice [B]: show the two hands playing as they play one more tune on the '
 'saxophone.\n'
 'Choice [C]: begin to appear, showing the hands playing various notes on the '
 'screen.\n'
 'Choice [D]: flash and a keyboard appears with the word " connect " and an '
 'arrow going down.\n'
 'Which choice is the best answer?')

Correct Choice: C
Number of Correct Responses: 0/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


In [109]:
test_single_example_multiple_times(9, num_samples=10)

Activity: Smoking a cigarette
Question 10:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A camera pans up to a man sitting and smoking a cigarette. Another '
 'man walks over with a cart and begins digging through trash. the man smoking '
 'cigarettes\n'
 'Choice [A]: continues shoveling the debris along the ground.\n'
 'Choice [B]: walks back with the cart while the second man continues to '
 'shovel the dirt.\n'
 'Choice [C]: takes the cigarette out of his mouth and finally spits it out.\n'
 'Choice [D]: begins talking to the homeless man about how life sucks and how '
 'he can relate to the homeless man.\n'
 'Which choice is the best answer?')

Correct Choice: D
Number of Correct Responses: 10/10
Response Distribution:
A: 0.00%
B: 0.00%
C: 0.00%
D: 100.00%


# Testing Multiple Hyperparameter Profiles

In [122]:
# Define hyperparameter profiles
hyperparameter_profiles = [
    {"name": "most_conservative", "temperature": 0, "top_k": 50, "top_p": 0.95},
    {"name": "conservative", "temperature": 0.8, "top_k": 10, "top_p": 0.5},
    {"name": "default", "temperature": 0.8, "top_k": 40, "top_p": 0.9},
    {"name": "diverse", "temperature": 1.2, "top_k": 50, "top_p": 0.95},
    {"name": "most_diverse", "temperature": 1.5, "top_k": 50, "top_p": 0.95},
]

# Function to test a single example multiple times with different hyperparameters
def test_single_example_with_hyperparameters(idx, num_samples=10):
    example = sampled_df.iloc[idx]
    question, choices, label = format_example(example)
    prompt = "Your task is to think carefully through a reasoning problem.\n"
    prompt += "Respond only with [A], [B], [C], or [D]\n"
    prompt += f"Question: {question}\n"
    labels = ['A', 'B', 'C', 'D']
    for i, choice in enumerate(choices):
        prompt += f"Choice [{labels[i]}]: {choice}\n"
    prompt += "Which choice is the best answer?"

    results = []

    for profile in hyperparameter_profiles:
        llm = Ollama(model="llama3", temperature=profile["temperature"], top_k=profile["top_k"], top_p=profile["top_p"])
        responses = []
        for _ in range(num_samples):
            response = llm.invoke(prompt)
            predicted_choice = extract_choice(response)
            responses.append(predicted_choice)
        
        correct_responses = sum(1 for choice in responses if choice == label)
        response_distribution = {labels[i]: responses.count(i) for i in range(4)}

        results.append({
            "profile": profile["name"],
            "correct_responses": correct_responses,
            "total_samples": num_samples,
            "response_distribution": response_distribution,
        })

    clear_output(wait=True)
    print(f"Activity: {example['activity_label']}")
    print(f"Question {idx + 1}:")
    pprint(prompt)
    print(f"\nCorrect Choice: {labels[label]}")
    for result in results:
        print(f"\nProfile: {result['profile']}")
        print(f"Number of Correct Responses: {result['correct_responses']}/{result['total_samples']}")
        distribution = ""
        for label, count in result['response_distribution'].items():
            percentage = (count / num_samples) * 100
            distribution += f"{label}: {percentage:.0f}% "
        print("Response Distribution:", distribution)

        

N=50
# Test a single example by index with multiple hyperparameter profiles
test_single_example_with_hyperparameters(0, num_samples=N)  # Change the index to test different examples

Activity: Camel ride
Question 1:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man is seen holding a small child while looking and smiling to '
 'the camera. The two then climb on a camel and ride around while waving to '
 'the camera. the two\n'
 'Choice [A]: are shown in several more shots climbing on the camel and '
 'walking back.\n'
 'Choice [B]: end up getting off and walking around their ride.\n'
 'Choice [C]: continue riding until the baby pulls a dog down off the camel '
 'and the man leaves.\n'
 'Choice [D]: ride back together with the woman leading them in front.\n'
 'Which choice is the best answer?')

Correct Choice: D

Profile: most_conservative
Number of Correct Responses: 50/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: conservative
Number of Correct Responses: 50/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: default
Number of Correct Responses: 47/50
Respons

We can see that the conservative profiles perform best in selecting the correct choice 'ride back together with the woman leading them in front'. As diversity hyperparameters increase, the second most selected choice is 'continue riding until the baby pulls a dog down off the camel and the man leaves'. The other choices do not get selected as they make less sense in context, as in the case of choice A where they continue climbing on and walking, or choice B where they get off and walk around. This suggests the models are still using their world knowledge to select the most likely responses. Notably, the hyperparameter settings significantly impact the model's output, with lower temperatures and top_k values favoring more predictable and contextually accurate responses, while higher values introduce greater response diversity. This balance between consistency and creativity highlights the model's ability to generate safe, accurate answers in conservative settings and explore wider possibilities in more diverse profiles.

In [123]:
test_single_example_with_hyperparameters(1, num_samples=N)

Activity: Drum corps
Question 2:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Text appears on the screen with a link. People in uniform march '
 'down the street. it\n'
 'Choice [A]: then cuts to a news anchor with a laptop.\n'
 'Choice [B]: ends with a lead in footage of the participants walking and '
 'cheering.\n'
 'Choice [C]: cuts to the crowd looking on.\n'
 'Choice [D]: screens an ad for the snow camp, with its picture of a cabin.\n'
 'Which choice is the best answer?')

Correct Choice: C

Profile: most_conservative
Number of Correct Responses: 0/50
Response Distribution: A: 100% B: 0% C: 0% D: 0% 

Profile: conservative
Number of Correct Responses: 17/50
Response Distribution: A: 66% B: 0% C: 34% D: 0% 

Profile: default
Number of Correct Responses: 8/50
Response Distribution: A: 60% B: 14% C: 16% D: 10% 

Profile: diverse
Number of Correct Responses: 12/50
Response Distribution: A: 42% B: 16% C: 24% 

The conservative profiles struggled significantly with this question, failing to select the correct choice 'C' consistently. The most_conservative profile selected 'A' exclusively, suggesting a misalignment with the context of a marching band performance. As diversity in the profiles increased, there was a gradual improvement in selecting 'C', though choices 'A' and 'D' remained prevalent. This scenario highlights how conservative settings may lead to overfitting on less probable, yet internally consistent, narratives, while more diverse settings introduce variety that, in this case, edges closer to the correct context. The interplay of hyperparameters here underscores the balance needed between strict adherence to context and creative exploration to improve accuracy in nuanced scenarios.

In [124]:
test_single_example_with_hyperparameters(2, num_samples=N)

Activity: Slacklining
Question 3:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A man stretches a rope between two trees. he\n'
 'Choice [A]: gets on the rope and prepares to start a race.\n'
 'Choice [B]: then balances on the rope as he walks, sits, and bounces on it.\n'
 'Choice [C]: begins walking forward in the path.\n'
 'Choice [D]: puts a shoe on to hold onto the rope.\n'
 'Which choice is the best answer?')

Correct Choice: B

Profile: most_conservative
Number of Correct Responses: 0/50
Response Distribution: A: 0% B: 0% C: 100% D: 0% 

Profile: conservative
Number of Correct Responses: 21/50
Response Distribution: A: 0% B: 42% C: 58% D: 0% 

Profile: default
Number of Correct Responses: 19/50
Response Distribution: A: 0% B: 38% C: 42% D: 20% 

Profile: diverse
Number of Correct Responses: 16/50
Response Distribution: A: 4% B: 32% C: 36% D: 28% 

Profile: most_diverse
Number of Correct Responses: 15/50


In this scenario, the most_conservative profile incorrectly selected 'C' exclusively, indicating a rigid interpretation of the context. The conservative profile showed some improvement, but still favored 'C' over 'B'. The default and diverse profiles demonstrated more balanced distributions, with 'B' being selected more frequently as diversity increased. This pattern suggests that while conservative profiles tend to overfit on less probable, linear narratives, more diverse profiles introduce necessary variability that can capture more contextually appropriate actions. This highlights the importance of tuning hyperparameters to balance between context adherence and creative response generation.

In [125]:
test_single_example_with_hyperparameters(3, num_samples=N)

Activity: Computers and Electronics
Question 4:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to change a computer hard drive disk [title] If your '
 'hard drive is still working, make sure you make recovery disk or image by '
 'using backup or recovery software. [step] If your drive has already failed, '
 'hopefully you have your important files saved somewhere as backup. [title] '
 'Obtain a new hard drive.\n'
 'Choice [A]: [step] Most small-sized hard drives need an old hard drive to go '
 'into recovery. However, it may take a long time to get old hard drives to '
 'get back to their original capacity.\n'
 'Choice [B]: [step] Make sure you have the right type of hard drive for your '
 'computer. Generally speaking, you want to replace your original drive with a '
 'similar drive to make the process easier and simpler.\n'
 'Choice [C]: [step] An external hard drive, such as a wineing or tonneau

In this straightforward technical scenario, all profiles consistently selected the correct choice 'B', demonstrating the model's strong grasp of logical and factual content. The uniformity across all profiles indicates that certain contexts, particularly those grounded in clear procedural knowledge, are handled well by the model regardless of hyperparameter settings. This uniformity highlights how some questions benefit from conservative approaches where predictability and adherence to factual accuracy are paramount. The lack of variation in response distributions also suggests that in well-defined scenarios, the model’s accuracy remains robust across different settings.

In [126]:
test_single_example_with_hyperparameters(4, num_samples=N)

Activity: Horseback riding
Question 5:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: . the lady in blue vest and brown jacket\n'
 'Choice [A]: has some more practice punches and then takes one hand grab her '
 'elbow and go straight into a crouch and to the side to do the same and then '
 'grab her armpit to throw the punch.\n'
 'Choice [B]: is sitting on a brown horse, while a man is standing beside the '
 'horse, holding another horse.\n'
 'Choice [C]: holds an orange handled knife and the lady in the green vest '
 'holds an orange handled knife with the blue handled handle.\n'
 'Choice [D]: is now standing at the mountain top preparing to get started '
 'putting bungee jumps on the ground.\n'
 'Which choice is the best answer?')

Correct Choice: B

Profile: most_conservative
Number of Correct Responses: 0/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: conservative
Number of Correct Responses

In this scenario, the most_conservative profile failed to select the correct choice 'B', instead opting exclusively for 'D', which suggests an inaccurate understanding of the context. The conservative and default profiles showed improvement, with 'B' being selected more frequently. As diversity increased, the distribution of responses became more varied, with choices 'A' and 'D' being selected alongside 'B'. This highlights that while conservative settings may overfit on less plausible narratives, increasing diversity allows the model to explore a broader range of possible actions, thereby increasing the likelihood of selecting contextually appropriate responses. The varied performance underscores the need to balance between strict context adherence and creative exploration to improve accuracy.

In [127]:
test_single_example_with_hyperparameters(5, num_samples=N)

Activity: Building sandcastles
Question 6:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A group of kids is building a moat filled with water around a sand '
 'castle. A little girl with floaters on pour water in the moat. we\n'
 'Choice [A]: see fireworks going off in the background as the kids continue '
 'to build.\n'
 'Choice [B]: see an opening title screen.\n'
 'Choice [C]: see children on the other side avoiding the water castle.\n'
 'Choice [D]: see kids across the beach working on their castles in the wet '
 'sand.\n'
 'Which choice is the best answer?')

Correct Choice: D

Profile: most_conservative
Number of Correct Responses: 50/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: conservative
Number of Correct Responses: 50/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: default
Number of Correct Responses: 45/50
Response Distribution: A: 0% B: 0% C: 10% D: 90% 

Profile: 

In this scenario, all profiles except the diverse and most_diverse consistently selected the correct choice 'D', indicating a strong contextual understanding by the model. The most_conservative and conservative profiles performed flawlessly, suggesting that predictable settings help maintain accuracy. The diverse profiles, while still largely accurate, introduced more variation with choices 'C', indicating a shift towards broader narrative exploration. This demonstrates that while conservative profiles excel in clear, predictable contexts, introducing diversity can still yield accurate responses while allowing for creative variability, underscoring the need for balanced hyperparameter tuning to maintain contextually appropriate responses.

In [128]:
test_single_example_with_hyperparameters(6, num_samples=N)

Activity: Putting on makeup
Question 7:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: Young girl puts black mascara on her eyelashes. she\n'
 'Choice [A]: then touches her eyelids, her eyebrows, and her cheeks.\n'
 'Choice [B]: then applies painted and painted eyeliner to her eye lashes.\n'
 'Choice [C]: looks into the large mirror.\n'
 'Choice [D]: puts red colored eye lashes, then does eye shadow.\n'
 'Which choice is the best answer?')

Correct Choice: C

Profile: most_conservative
Number of Correct Responses: 0/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: conservative
Number of Correct Responses: 0/50
Response Distribution: A: 38% B: 0% C: 0% D: 62% 

Profile: default
Number of Correct Responses: 9/50
Response Distribution: A: 20% B: 14% C: 18% D: 48% 

Profile: diverse
Number of Correct Responses: 5/50
Response Distribution: A: 30% B: 18% C: 10% D: 42% 

Profile: most_diverse
Number of C

The most_conservative profile failed to select the correct choice 'C', instead opting exclusively for 'D'. The conservative profile also struggled, showing a preference for 'D' and 'A'. The default and diverse profiles demonstrated more balanced distributions, with 'C' being selected more frequently as diversity increased. This pattern suggests that conservative profiles may overfit on less plausible narratives, while more diverse profiles introduce necessary variability that can capture more contextually appropriate actions. This highlights the importance of tuning hyperparameters to balance between context adherence and creative response generation, ensuring that the model can adapt to different scenarios effectively.

In [129]:
test_single_example_with_hyperparameters(7, num_samples=N)

Activity: Work World
Question 8:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: [header] How to explain a termination in a job interview [title] '
 'Be honest. [step] When your interviewer asks you why you left your last job, '
 'the best thing you can do is be honest. Making up stories will only make you '
 'seem irresponsible and untrustworthy.\n'
 'Choice [A]: If you can just paint a picture of the situation, you will '
 'hardly convince your interviewer. Stick to facts you can use to bridge the '
 'gap between your actions and those of the employer.\n'
 "Choice [B]: Don't exaggerate what might have happened or be vague about the "
 "specific circumstances that led to you leaving. [substeps] Don't contact one "
 "of your supervisors to tell them you've terminated your job.\n"
 'Choice [C]: You will seem like a liar by having an inaccurate job '
 'description and a reputation of being a failure. Choose facts 

In this job interview scenario, the conservative profiles performed flawlessly, consistently selecting the correct choice 'D'. The default and diverse profiles showed more variability, with choices 'B' and 'D' being selected equally often. This indicates that while conservative settings maintain a strong adherence to factual accuracy and context, introducing diversity can lead to broader narrative exploration. However, the presence of a clear, factual context allows conservative profiles to excel, highlighting their strength in scenarios where predictability and adherence to logical consistency are crucial. This consistency underscores the importance of balanced hyperparameter tuning to maintain accuracy while allowing for creative variability when needed.

In [130]:
test_single_example_with_hyperparameters(8, num_samples=N)

Activity: Playing saxophone
Question 9:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A magazine about saxophones is shown just before a pair of hands '
 'are shown playing a saxophone. multiple screens\n'
 'Choice [A]: appear with features to replace the ones shown on the tv '
 'screen.\n'
 'Choice [B]: show the two hands playing as they play one more tune on the '
 'saxophone.\n'
 'Choice [C]: begin to appear, showing the hands playing various notes on the '
 'screen.\n'
 'Choice [D]: flash and a keyboard appears with the word " connect " and an '
 'arrow going down.\n'
 'Which choice is the best answer?')

Correct Choice: C

Profile: most_conservative
Number of Correct Responses: 0/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: conservative
Number of Correct Responses: 0/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: default
Number of Correct Responses: 8/50
Response Distribu

In this scenario, the conservative profiles failed to select the correct choice 'C', opting exclusively for 'D'. The default and diverse profiles showed some improvement, with 'C' being selected more frequently. This pattern suggests that conservative profiles may overfit on less plausible narratives, while more diverse profiles introduce necessary variability that can capture more contextually appropriate actions. This highlights the importance of tuning hyperparameters to balance between context adherence and creative response generation, ensuring that the model can adapt to different scenarios effectively. The varied performance underscores the need for balanced hyperparameter settings to maintain accuracy while allowing for creative exploration in more nuanced contexts.

In [131]:
test_single_example_with_hyperparameters(9, num_samples=N)

Activity: Smoking a cigarette
Question 10:
('Your task is to think carefully through a reasoning problem.\n'
 'Respond only with [A], [B], [C], or [D]\n'
 'Question: A camera pans up to a man sitting and smoking a cigarette. Another '
 'man walks over with a cart and begins digging through trash. the man smoking '
 'cigarettes\n'
 'Choice [A]: continues shoveling the debris along the ground.\n'
 'Choice [B]: walks back with the cart while the second man continues to '
 'shovel the dirt.\n'
 'Choice [C]: takes the cigarette out of his mouth and finally spits it out.\n'
 'Choice [D]: begins talking to the homeless man about how life sucks and how '
 'he can relate to the homeless man.\n'
 'Which choice is the best answer?')

Correct Choice: D

Profile: most_conservative
Number of Correct Responses: 50/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: conservative
Number of Correct Responses: 50/50
Response Distribution: A: 0% B: 0% C: 0% D: 100% 

Profile: default
Number of 

In this scenario, the conservative profiles again showed their strength, correctly identifying choice 'D' as the most appropriate response, reflecting a nuanced understanding of the social dynamics at play. The default profile showed some variability, introducing 'C' and 'B' as secondary choices, which though plausible, lack the deeper narrative connection present in 'D'. The diverse profiles exhibited even more variation, with choices 'A' and 'B' emerging, indicating a shift towards introducing new, but less contextually fitting, elements into the story. This highlights the balance the model strikes between creativity and accuracy—while conservative settings yield highly probable and contextually accurate answers, more diverse settings explore alternative narratives that, while imaginative, may stray from the expected storyline. These patterns underscore the impact of hyperparameter settings on model behavior, with lower temperatures and top_k values enhancing consistency and contextual accuracy, and higher values promoting creative, yet less accurate, responses.